In [39]:
!pip install flask flask-cors opencv-python-headless pytesseract pandas flask-ngrok
!apt install tesseract-ocr
!apt-get install -y libsm6 libxext6 libxrender-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 56 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsm6 is already the newest version (2:1.2.3-1build2).
libxext6 is already the newest version (2:1.3.4-1build1).
libxrender-dev is already the newest version (1:0.9.10-1build4).
0 upgraded, 0 newly installed, 0 to remove and 56 not upgraded.


In [63]:
!pip install flask flask-ngrok
!wget https://bin.equinox.io/c/4b9b1b9c72f0/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip


--2024-09-05 14:50:39--  https://bin.equinox.io/c/4b9b1b9c72f0/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 54.161.241.46, 52.202.168.65, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2024-09-05 14:50:40 ERROR 404: Not Found.

unzip:  cannot find or open ngrok-stable-linux-amd64.zip, ngrok-stable-linux-amd64.zip.zip or ngrok-stable-linux-amd64.zip.ZIP.


In [64]:
import logging
logging.basicConfig(level=logging.INFO)

In [65]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from werkzeug.utils import secure_filename
import os
import cv2
import pytesseract
import pandas as pd
from flask_ngrok import run_with_ngrok

In [66]:
app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

In [67]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [68]:
medicine_df = pd.read_csv('/content/A_Z_medicines_dataset_of_India.csv')
medicine_names = medicine_df['name'].str.lower().tolist()

In [69]:
UPLOAD_FOLDER = os.path.join('demo')
if not os.path.isdir(UPLOAD_FOLDER):
    os.mkdir(UPLOAD_FOLDER)

In [70]:
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO)

# Example usage in routes
@app.route("/upload/lab-report", methods=["POST"])
def upload_lab_report():
    response = []

    if 'file' not in request.files:
        logging.error("No file part in the request")
        return jsonify({"message": "No file part in the request"}), 400

    files = request.files.getlist("file")

    for file in files:
        if file.filename == '':
            logging.error("No selected file")
            return jsonify({"message": "No selected file"}), 400

        filename = secure_filename(file.filename)
        destination = os.path.join(UPLOAD_FOLDER, filename)
        file.save(destination)

        try:
            extracted_text = extract_text_from_image(destination)
            detected_medicines = find_medicine_names(extracted_text)
        except Exception as e:
            logging.error(f"Error processing file {filename}: {e}")
            continue

        response.append({
            "file": filename,
            "extracted_medicines": detected_medicines
        })

        os.remove(destination)

    return jsonify({"message": response})


In [71]:
def extract_text_from_image(image_path):
    """ Extracts text from an image using Tesseract OCR """
    # Read the image
    img = cv2.imread(image_path)

    # Convert the image to grayscale (optional for better OCR results)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Perform OCR on the grayscale image
    extracted_text = pytesseract.image_to_string(gray)

    return extracted_text.lower()

In [72]:
def find_medicine_names(extracted_text):
    """ Find medicine names from extracted text based on the loaded dataset """
    detected_medicines = []

    # Split the extracted text into words
    words_in_text = extracted_text.split()

    # Check if any word in the text matches a medicine name
    for word in words_in_text:
        if word in medicine_names:
            detected_medicines.append(word)

    return detected_medicines

In [77]:
if __name__ == "__main__":
    app.run()

Ibuprofen
